In [1]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from src.GGUFBot import GGUFBot
from src.HFDataset import HFDataset
from src.SimpleQuestionGenerator import SimpleQuestionGenerator
from src.AnswerGenerator import AnswerGenerator
import pandas as pd
import random
import json
from src.GGUFEvaluator import prepare_prompt,parse_output,GGUFEvaluator

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_path="/home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf"
max_new_tokens=4000
n_layers=400
bot = GGUFBot(model_path, max_new_tokens=max_new_tokens, n_ctx=max_new_tokens, n_gpu_layers=n_layers)

llama_model_loader: additional 3 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 29 key-value pairs and 563 tensors from /home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mixtral-8x22B-Inst...
llama_model_loader: - kv   2:                          llama.block_count u32              = 56
llama_model_loader: - kv   3:                       llama.context_length u32              = 65536
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 6144
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 16384
llama_model_loader: - 

llama_model_loader: - kv  17:                      tokenizer.ggml.scores arr[f32,32768]   = [-1000.000000, -1000.000000, -1000.00...
llama_model_loader: - kv  18:                  tokenizer.ggml.token_type arr[i32,32768]   = [3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  19:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  20:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  21:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  22:               tokenizer.ggml.add_bos_token bool             = false
llama_model_loader: - kv  23:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  24:                    tokenizer.chat_template str              = {{bos_token}}{% for message in messag...
llama_model_loader: - kv  25:               general.quantization_version u32              = 2
llama_model_loader: - kv  26:

loading model...


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   yes
ggml_cuda_init: CUDA_USE_TENSOR_CORES: no
ggml_cuda_init: found 8 CUDA devices:
  Device 0: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 1: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 2: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 3: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 4: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 5: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 6: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 7: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
llm_load_tensors: ggml ctx size =    2.51 MiB
llm_load_tensors: offloading 56 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 57/57 layers to GPU
llm_load_tensors:        CPU buffer size =   132.00 MiB
llm_load_tensors:      CUDA0 buffer size = 12238.73 MiB
llm_load_tenso

In [8]:
txt="""次の文章を､日本の文化に即した平易な文章に書き直してください
#文章
次の記事を読んで、最も適切な回答を選択してください。記事: 昔ながらの遊び - 有料の子供向け!ボールを飲みに来てください！あるいは60,000個！アメリカ全土のショッピングモールや近所に、おもちゃやゲーム、ガイド付きの楽しみ、家計に負担をかけないトレーニングなどを特徴とする、従量制の屋内遊び場を提供する新しいタイプのビジネスフランチャイズが登場している。公共の遊び場がますます磨耗して汚くなる中、営利センターでは、清潔で安全なガイド付きアクティビティや、子供の体力を伸ばすためのさまざまなやりがいのあるエクササイズを、通常は 1 時間あたり約 5 ドルの料金で提供しています。カンザスシティに本拠を置くチェーン店、ニューヨーク州ヨンカーズにある設立2か月のディスカバリー・ゾーンのオーナー、ディック・グッゲンハイマー氏は、「遊び場は汚いし、案内もされていない」と語る。安全です」。共働き世帯のニーズを満たすために、この新しいフランチャイズは、従来の公共の遊び場が暗くなって使用できなくなってからもずっと夜も営業しています。ただし、これらの新しい遊び場は保育所を目的としたものではありません。親は子供を送り出すのではなく、家にいて一緒に遊ぶことが期待されています。しかし、ハイテクベビーシッターサービスを提供しているところもあります。ディスカバリー ゾーンの一部では、親が子供を特別なガイド付きプログラムに登録した後、子供を残して数時間こっそりと映画やディナーを楽しむことができます。何か問題があれば、お父さんとお母さんが呼ばれます。しかし、何よりも楽しいのは、共働き家庭で通勤に 2 時間かかる以前の時代に親がやっていたようなこと、子供たちと遊ぶことができるようになることです。少なくとも、時間当たりの料金を考えても、これは時代遅れです。質問: この記事は主に何について話していますか? - お子様は親の世話なしで遊ぶことができます。 - ディスカバリーゾーンの急速な発展。 - 新しいタイプのキッズビジネスフランチャイズ。 - 屋外遊び場の欠点。 この質問に対する答えは次のとおりです
#書き直した文章
"""
txt="""step by stepで､日本語で回答してください
タスクは、与えられた数学の問題に対する正しい答えを生成することです。 Q: 問題: 金は水の 19 倍、銅は水の 9 倍の重さです。水の15倍の重さの合金を得るには、これらをどのような割合で混合すればよいでしょうか? 答え: """
r=bot.ask(txt)
r

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2985.50 ms
llama_print_timings:      sample time =     114.43 ms /   534 runs   (    0.21 ms per token,  4666.65 tokens per second)
llama_print_timings: prompt eval time =    1300.28 ms /   135 tokens (    9.63 ms per token,   103.82 tokens per second)
llama_print_timings:        eval time =  100137.71 ms /   533 runs   (  187.88 ms per token,     5.32 tokens per second)
llama_print_timings:       total time =  102610.96 ms /   668 tokens


' この問題を解くためには、比例の原理を用います。\n\n1. 金が水の19倍の重さであることから、金の量が1つの単位（たとえば、グラム）であれば、水は1/19単位に相当します。\n2. 銅が水の9倍の重さであることから、銅の量が1つの単位（たとえば、グラム）であれば、水は1/9単位に相当します。\n3. 合金を得るには、水の15倍の重さにならなくてはなりませんが、金と銅の比例割合は変わりません。それぞれの要素の量を決めるために、比例の原理を用いることができます。\n4. 次のような方程式を立てます：$x \\cdot 19 + y \\cdot 9 = 15$ (ここで、$x$は金の量、$y$は銅の量を表します)。\n5. この方程式を解くために、$y = \\frac{15 - 19x}{9}$ と変形します。\n6. $x$と$y$の両者が整数である必要があることから、$x=3$と$y=2$を選びます（他にも解は存在しますが、この例では$x=3$, $y=2$が最も簡単な解です）。\n7. よって、金と銅の比例割合は3:2です。つまり、合金を得るには、金を水の3倍（19 \\* 3 = 57倍）と銅を水の2倍（9 \\* 2 = 18倍）の重さ分混合する必要があります。'

In [9]:
from datasets import load_dataset
ds=load_dataset("atsushi3110/cross-lingual-openorcha-830k-en-ja",split="train")
df=pd.DataFrame(ds)
df["database"]="atsushi3110/cross-lingual-openorcha-830k-en-ja_"+df["id/en"]
df["question"]=df["question/ja"]
df=df.drop(columns=["response/en","system_prompt/en"],inplace=True)
df[:3]

In [16]:
from datasets import load_dataset
import random


ds_name="hatakeyama-llm-team/WikiBookJa"
ds=load_dataset(ds_name,split="train")

In [43]:
inst_list=[
"回答文は丁寧であること"
"回答文は簡潔であること"
"回答文はstep by stepで作文してください",
]

def gen_prompt(inst,text):
    prompt_template=f"""次の文章をもとに､日本語の質問文と日本語の回答文をそれぞれ一つ生成しなさい
    #制約
    - {inst}
    #文章
    {text}
    #質問文と回答文
    """
    return prompt_template

In [44]:
record=ds[random.randint(0,len(ds))]
prompt=gen_prompt(random.choice(inst_list)[:1000],record["text"])
r=bot.ask(prompt)

Llama.generate: prefix-match hit



llama_print_timings:        load time =    1630.62 ms
llama_print_timings:      sample time =      55.26 ms /   257 runs   (    0.22 ms per token,  4650.74 tokens per second)
llama_print_timings: prompt eval time =    5278.59 ms /  1227 tokens (    4.30 ms per token,   232.45 tokens per second)
llama_print_timings:        eval time =   27329.40 ms /   256 runs   (  106.76 ms per token,     9.37 tokens per second)
llama_print_timings:       total time =   33111.03 ms /  1483 tokens


In [46]:
q,a=r.split("回答文")
q=q.replace("質問文","").strip()
a=a.strip()
if a[0]=="：":
    a=a[1:]
if q[0]=="：":
    q=q[1:]
if a[0]==":":
    a=a[1:]
if q[0]==":":
    q=q[1:]

a=a.strip()
q=q.strip()
q,a

('日本の民法では、利息を元本に組入れることができる条件となっていますか？',
 'はい、日本の民法においては、利息を元本に組入れることができる条件が定められています。具体的には、債権者が催告をしても、債務者がその利息を支払わなかった場合に、債権者は当該利息債権を元本に組み入れることができます。ただし、1年以上の不払いと催告後の不払いが条件となり、単利計算が原則であることから、手続きを定めた規準としての存在意義は薄いものと考えられます。')

In [42]:
r

' 質問: この場所にはどんな種類の情報が集まっていますか？\n回答: この書庫には、音楽に関する様々な種類の情報が収録されています。これらの情報は、文書や資料、教科書等で提供されています。\n\n質問: 収録内容を具体的に教えてください。\n回答: 申し訳ありませんが、この文章からは具体的な収録内容について情報を提供できません。書庫内の音楽に関する各種情報を確認するためには、書庫に直接アクセスしてください。'